# Part 1 - Loading and Formatting Data

TODO: description of this notebook

### Upon completing this notebook, we'll know more about:  
* Using [Pandas] for data management
* Basics of web scraping

[Pandas]: https://pandas.pydata.org/

In [1]:
import os
import re
import time
import glob
import zipfile
import urllib.request
import pandas as pd

# Let's start by loading the data and have a peek at the contents 
The data was scraped from [zillow.com](https://www.zillow.com/homes/for_sale/San-Francisco-CA/) and is dispersed between several csv files.  
You can find the code used for scraping in [Github](https://github.com/kylehounslow/datasets/blob/master/mlbootcamp/scrape_zillow.ipynb)

We will first download and extract the [dataset](https://github.com/kylehounslow/datasets/blob/master/mlbootcamp/sf_housing.zip), then use Pandas to load all csv files and concatenate them into a single DataFrame

In [2]:
def download_extract_data(data_dir: str):
    """
    Download zipfile and extract its contents to data_dir
    Args:
        data_dir: extract data here
    """
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    data_save_path = os.path.join(data_dir, 'data.zip')
    data_download_url = 'https://github.com/kylehounslow/datasets/raw/master/mlbootcamp/sf_housing.zip'
    print('downloading data...')
    urllib.request.urlretrieve(data_download_url, data_save_path)
    print('extracting data to {data_dir}...'.format(data_dir=data_dir))
    data_zip = zipfile.ZipFile(data_save_path)
    data_zip.extractall(data_dir)
    data_zip.close()
    print('Done.')

def load_data_to_dataframe(data_dir: str) -> pd.DataFrame:
    """
    Load all .csv files from data_dir and concatenate them into a single DataFrame.  
    Args:
        data_dir: path to data directory
    Returns:
        pd.DataFrame: all data from files in data_dir
    Notes:
        All duplicate rows will be discarded
    """
    all_csvs = []
    # load the csv files from all scraping runs
    csv_filenames = os.path.join(data_dir, '**/*.csv')
    print('loading data {csv_filenames}'.format(csv_filenames=csv_filenames))
    for filename in glob.glob(csv_filenames):
        all_csvs.append(pd.read_csv(filename))
    # combine all dataframes together and drop any duplicate entries
    df = pd.concat(all_csvs, ignore_index=True).drop_duplicates()
    print("Found a total of {count} data points".format(count=len(df)))
    # save this combined dataframe as csv for safe keeping
    df.to_csv(os.path.join(data_dir, 'all_data.csv'), index=False)
    return df

Download and extract the data 

In [3]:
data_dir = './data'
download_extract_data(data_dir=data_dir)

downloading data...
extracting data to ./data...
Done.


The data is organized into multiple folders, each containing multiple .csv files from that scraping run

In [4]:
%%bash -s "$data_dir"
ls $1 -la

total 1160
drwxr-xr-x 8 root root   4096 Jun 17 03:57 .
drwxrwxr-x 4 1000 1000   4096 Jun 17 04:39 ..
-rw-r--r-- 1 root root 265245 Jun 17 04:39 all_data.csv
-rw-r--r-- 1 root root 317892 Jun 17 03:57 data.csv
-rw-r--r-- 1 root root 246700 Jun 17 04:40 data.zip
-rw-r--r-- 1 root root 317892 Jun 17 04:40 data_w_latlng.csv
drwxr-xr-x 2 root root   4096 Jun 17 03:07 feb28_2018
drwxr-xr-x 2 root root   4096 Jun 17 03:07 mar10_2018
drwxr-xr-x 2 root root   4096 Jun 17 03:07 mar5_2018
drwxr-xr-x 2 root root   4096 Jun 17 03:07 mar6_2018
drwxr-xr-x 2 root root   4096 Jun 17 03:07 mar9_2018
drwxr-xr-x 2 root root   4096 Jun 17 03:07 may11_2018


In [5]:
%%bash -s "$data_dir"
ls $1/may11_2018

properties-94102.csv
properties-94103.csv
properties-94104.csv
properties-94105.csv
properties-94107.csv
properties-94108.csv
properties-94109.csv
properties-94110.csv
properties-94111.csv
properties-94112.csv
properties-94114.csv
properties-94115.csv
properties-94116.csv
properties-94117.csv
properties-94118.csv
properties-94121.csv
properties-94122.csv
properties-94123.csv
properties-94124.csv
properties-94126.csv
properties-94127.csv
properties-94129.csv
properties-94131.csv
properties-94132.csv
properties-94133.csv
properties-94134.csv
properties-94139.csv
properties-94143.csv
properties-94146.csv
properties-94151.csv
properties-94158.csv
properties-94159.csv
properties-94177.csv
properties-94188.csv


Load all data into DataFrame and preview the contents using `head()`

In [6]:
df_raw = load_data_to_dataframe(data_dir=data_dir)
df_raw.head(5)  # display first 5 entries

loading data ./data/**/*.csv
Found a total of 1262 data points


,title,address,city,state,postal_code,price,facts and features,real estate provider,url
0,New Construction,288 Pacific Ave # 67UQRR,San Francisco,CA,94111,NaN,"-- bds , -- ba , -- sqft",NaN,https://www.zillow.com/community/288-pacific/2...
1,Condo For Sale,733 Front St UNIT 606,SAN FRANCISCO,CA,94111,"$1,195,000","1 bd , 1 ba , 920 sqft",Climb Real Estate,https://www.zillow.com/homedetails/733-Front-S...
2,Condo For Sale,101 Lombard St APT 401W,SAN FRANCISCO,CA,94111,"$1,425,000","2 bds , 2 ba , 1,238 sqft",Paragon Real Estate Group,https://www.zillow.com/homedetails/101-Lombard...
3,Condo For Sale,111 Chestnut St UNIT 109,SAN FRANCISCO,CA,94111,"$1,600,000","2 bds , 2 ba , 1,925 sqft",CENTURY 21 Real Estate Alliance,https://www.zillow.com/homedetails/111-Chestnu...
4,Lot/Land For Sale,1235 Sansome St,San Francisco,CA,94111,"$998,000","6,882 sqft lot",NaN,https://www.zillow.com/homedetails/1235-Sansom...


Our data is now contained in a variable named `df_raw` which is a pandas DataFrame.

### Display some quick stats about the DataFrame
DataFrame has a few built in functions we can call to get a quick summary of the data:  
* `info()` displays a count of all non-null objects and their datatypes  
* `describe()` calculates basic statistics about all numerical values in the DataFrame

In [7]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1262 entries, 0 to 2413
Data columns (total 9 columns):
title                   1262 non-null object
address                 1262 non-null object
city                    1262 non-null object
state                   1262 non-null object
postal_code             1262 non-null object
price                   1260 non-null object
facts and features      1262 non-null object
real estate provider    986 non-null object
url                     1262 non-null object
dtypes: object(9)
memory usage: 98.6+ KB


In [8]:
df_raw.describe()

,title,address,city,state,postal_code,price,facts and features,real estate provider,url
count,1262,1262,1262,1262,1262,1260,1262,986,1262
unique,11,1197,9,1,33,557,1028,199,1200
top,Condo For Sale,1214 Masonic Ave,San Francisco,CA,94109,"$1,495,000","2 bds , 1 ba , -- sqft",Zephyr Real Estate,https://www.zillow.com/homedetails/1214-Masoni...
freq,559,3,863,1262,83,31,14,98,3


# Data Parsing
Many of the columns are not in a format ready for training a Machine Learning model. We need to parse the data from string values into numerical values.  
For instance the `price` columns contain `$` and `,` characters and the `facts and features` column is a written sentence describing the number of beds, bath and square footage. 
  
Let's parse and format these columns.

In [9]:
# copy our original dataframe for safe keeping. We will manipulate `df` instead
df = df_raw.copy()

### Reformat price column
Time to use some convenient Pandas functions such as `.apply()` to apply a user defined formatting function to all values in a column.  

Remove `$` and `,` characters and format as `int`.  
Also, some prices are represented as `$1K` and `$1M` so let's replace with `1000` and `1000000`

Define the price parsing function:

In [10]:
def format_price(price: str):
    """
    Remove all non-numerical values from price column
    Args:
        price: string representation of price
    Returns:
        price_num: floating point representation of price
    """
    price = str(price)
    multiply_factor = 1
    if 'M' in price:
        multiply_factor = 1e6
    elif 'K' in price:
        multiply_factor = 1e3
    non_decimal = re.compile(r'[^0-9\.]')
    price_num = None
    try:
        price_num = float(non_decimal.sub('', price))*multiply_factor
    except Exception as e:
#         print(f'error converting \"{price}\": {e}')
        pass
    finally:
        return price_num

We can see the effect of `format_price()` when applied to a single value

In [11]:
price_str = df.price[5]
price_float = format_price(price_str)
print("string: {price}".format(price=price_str))
print("float: {price}".format(price=price_float))

string: $678,888
float: 678888.0


Apply `format_price()` to all values in the `price` column

In [12]:
df['price'] = df.price.apply(format_price)

In [13]:
df.head()

,title,address,city,state,postal_code,price,facts and features,real estate provider,url
0,New Construction,288 Pacific Ave # 67UQRR,San Francisco,CA,94111,NaN,"-- bds , -- ba , -- sqft",NaN,https://www.zillow.com/community/288-pacific/2...
1,Condo For Sale,733 Front St UNIT 606,SAN FRANCISCO,CA,94111,1195000.0,"1 bd , 1 ba , 920 sqft",Climb Real Estate,https://www.zillow.com/homedetails/733-Front-S...
2,Condo For Sale,101 Lombard St APT 401W,SAN FRANCISCO,CA,94111,1425000.0,"2 bds , 2 ba , 1,238 sqft",Paragon Real Estate Group,https://www.zillow.com/homedetails/101-Lombard...
3,Condo For Sale,111 Chestnut St UNIT 109,SAN FRANCISCO,CA,94111,1600000.0,"2 bds , 2 ba , 1,925 sqft",CENTURY 21 Real Estate Alliance,https://www.zillow.com/homedetails/111-Chestnu...
4,Lot/Land For Sale,1235 Sansome St,San Francisco,CA,94111,998000.0,"6,882 sqft lot",NaN,https://www.zillow.com/homedetails/1235-Sansom...


### Parse `facts and features` column into multiple columns 
An example entry in this column: `3 bds , 2 ba , 1,520 sqft`  
Parse the text using comma followed by a space '`, `' as the delimiter so that we can still capture the comma in the square footage.

Define parsing functions:

In [14]:
non_decimal = re.compile(r'[^\d.]+') # regex for removing non-decimal characters from string
def parse_beds(string):
    strings = string.lower().split(', ')
    num_beds = None
    for s in strings:
        if "bd" in s:
            try:
                num_beds = float(non_decimal.sub('', s))
            except Exception as e:
                pass
        # treat studio as 0 bedrooms
        elif "studio" in s.lower():
            num_beds = 0
        return num_beds

def parse_bath(string):
    strings = string.lower().split(', ')
    num_bath = None
    for s in strings:
        if "ba" in s:
            try:
                num_bath = float(non_decimal.sub('', s))
            except Exception as e:
                pass
            finally:
                return num_bath
def parse_sqft(string):
    strings = string.lower().split(', ')
    sqft = None
    for s in strings:
        if "ft" in s:
            try:
                sqft = float(non_decimal.sub('', s))
            except Exception as e:
                pass
            finally:
                return sqft

Apply parsing functions to columns:

In [15]:
df['bed'] = df['facts and features'].apply(parse_beds)
df['bath'] = df['facts and features'].apply(parse_bath)
df['sqft'] = df['facts and features'].apply(parse_sqft)

### Parse `title` column and create `property_type` column
The title of the posting contains some information we can parse. For instance we can map `'Condo For Sale'` --> `condo`

First let's see if there is a pattern to the titles:

In [16]:
print(df.title.unique())
print(df.title.value_counts())

['New Construction' 'Condo For Sale' 'Lot/Land For Sale' 'House For Sale'
 'Apartment For Sale' 'Co-op For Sale' 'Coming Soon' 'Foreclosure'
 'For Sale by Owner' 'Auction' 'Townhouse For Sale']
Condo For Sale        559
House For Sale        384
Apartment For Sale    186
New Construction       68
Coming Soon            23
Lot/Land For Sale      13
Auction                 9
For Sale by Owner       8
Co-op For Sale          7
Foreclosure             4
Townhouse For Sale      1
Name: title, dtype: int64


Looks like there is a limited amount of unique values, which is good! We can design our parser to catch most cases.  
We won't parse 'For Sale by Owner' since it is too vague

Define the parsing function:

In [17]:
# property types mapping
property_types = {'Condo For Sale': 'condo', 
                  'House For Sale': 'house', 
                  'Apartment For Sale': 'apartment', 
                  'New Construction': 'new',
                  'Foreclosure': 'foreclosure', 
                   'Lot/Land For Sale': 'lot', 
                  'Coming Soon': 'coming', 
                  'Co-op For Sale': 'coop',
                  'Auction': 'auction', 
                  'For Sale by Owner': None, 
                  'Townhouse For Sale': 'townhouse'}
def parse_property_type(string):
    try:
        property_type = property_types[string]
    except KeyError as e:
        print(e)
        property_type = None
    finally:
        return property_type

Apply parser to column:

In [18]:
df['property_type'] = df['title'].apply(parse_property_type)

In [19]:
df.property_type.value_counts()

condo          559
house          384
apartment      186
new             68
coming          23
lot             13
auction          9
coop             7
foreclosure      4
townhouse        1
Name: property_type, dtype: int64

That's it for parsing!  
Now if we check the `info()` of the dataframe, we should see some columns are now numerical (`float64`)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1262 entries, 0 to 2413
Data columns (total 13 columns):
title                   1262 non-null object
address                 1262 non-null object
city                    1262 non-null object
state                   1262 non-null object
postal_code             1262 non-null object
price                   1260 non-null float64
facts and features      1262 non-null object
real estate provider    986 non-null object
url                     1262 non-null object
bed                     1241 non-null float64
bath                    1075 non-null float64
sqft                    1171 non-null float64
property_type           1254 non-null object
dtypes: float64(4), object(9)
memory usage: 178.0+ KB


Call `describe()` to get some stats about the numerical values

In [21]:
df.describe()

,price,bed,bath,sqft
count,1.260000e+03,1241.000000,1075.000000,1171.000000
mean,2.020602e+06,2.130540,2.102791,2041.099915
std,2.215145e+06,1.554093,1.252006,1818.208577
min,2.000000e+05,0.000000,1.000000,1.000000
25%,9.359428e+05,1.000000,1.000000,1031.000000
50%,1.395000e+06,2.000000,2.000000,1510.000000
75%,2.100000e+06,3.000000,2.500000,2362.000000
max,2.500000e+07,12.000000,12.000000,17208.000000


## We have successfully loaded and formatted our dataset using Pandas. Now that we have the gist of the dataset contents, it's time to go more in depth and Visualize the data. See you in the next notebook!